In [1]:
import pandas as pd
import numpy as np
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import matplotlib.pyplot as plt

# Set precision
getcontext().prec = 30
scale_factor = Decimal("1")

# Base coordinates
trunk_w = 0.15
trunk_h = 0.2
base_w = 0.7
mid_w  = 0.4
top_w  = 0.25
tip_y = 0.8
tier_1_y = 0.5
tier_2_y = 0.25
base_y = 0.0
trunk_bottom_y = -trunk_h

BASE_COORDS = np.array([
    (0.0, tip_y),
    (top_w / 2, tier_1_y),
    (top_w / 4, tier_1_y),
    (mid_w / 2, tier_2_y),
    (mid_w / 4, tier_2_y),
    (base_w / 2, base_y),
    (trunk_w / 2, base_y),
    (trunk_w / 2, trunk_bottom_y),
    (-(trunk_w / 2), trunk_bottom_y),
    (-(trunk_w / 2), base_y),
    (-(base_w / 2), base_y),
    (-(mid_w / 4), tier_2_y),
    (-(mid_w / 2), tier_2_y),
    (-(top_w / 4), tier_1_y),
    (-(top_w / 2), tier_1_y),
])

class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = float(center_x)
        self.center_y = float(center_y)
        self.angle = float(angle)
        self._polygon = None

    @property
    def polygon(self):
        if self._polygon is None:
            rad = np.radians(self.angle)
            c, s = np.cos(rad), np.sin(rad)
            R = np.array([[c, -s], [s, c]])
            new_coords = BASE_COORDS @ R.T + np.array([self.center_x, self.center_y])
            self._polygon = Polygon(new_coords)
        return self._polygon

def parse_csv(csv_path):
    df = pd.read_csv(csv_path)
    df["x"] = df["x"].astype(str).str.strip().str.lstrip("s")
    df["y"] = df["y"].astype(str).str.strip().str.lstrip("s")
    df["deg"] = df["deg"].astype(str).str.strip().str.lstrip("s")
    df[["group_id", "item_id"]] = df["id"].str.split("_", n=2, expand=True)
    
    dict_of_tree_list = {}
    for group_id, group_data in df.groupby("group_id"):
        tree_list = [
            ChristmasTree(center_x=row["x"], center_y=row["y"], angle=row["deg"])
            for _, row in group_data.iterrows()
        ]
        dict_of_tree_list[int(group_id)] = tree_list
    return dict_of_tree_list

def calculate_score(csv_path):
    tree_dict = parse_csv(csv_path)
    total_score = 0
    
    for n, trees in tree_dict.items():
        all_polygons = [t.polygon for t in trees]
        bounds = unary_union(all_polygons).bounds
        width = bounds[2] - bounds[0]
        height = bounds[3] - bounds[1]
        side = max(width, height)
        total_score += (side ** 2) / n
        
    return total_score

# Calculate scores for both files
score_base = calculate_score("submission.csv")
score_sa = calculate_score("submission_sa_parallel.csv")

print(f"Baseline Score: {score_base}")
print(f"SA Score: {score_sa}")
print(f"Improvement: {score_base - score_sa}")


Baseline Score: 70.67610239809186
SA Score: 70.6761023980918
Improvement: 5.684341886080802e-14
